<a href="https://colab.research.google.com/github/DILIPKUMARJAULA/NLP/blob/main/Copy_of_Assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install Required Libraries

In [2]:
!pip install torch transformers


2. Import Libraries

In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random


3. Sample Text Data

In [5]:
# Sample text data (replace with a larger dataset if available)
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""


4. Define Dataset Class for Tokenization

In [6]:
class TextDataset(Dataset):
    def __init__(self, text, tokenizer, max_length=128):
        self.examples = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=max_length)

    def __len__(self):
        return self.examples['input_ids'].size(0)

    def __getitem__(self, i):
        return {key: val[i] for key, val in self.examples.items()}


5. Initialize Model and Tokenizer

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

6. Prepare Dataset and DataLoader

In [8]:
# Set the padding token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Prepare Dataset and DataLoader
dataset = TextDataset(text, tokenizer)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)


7. Define Training Function

In [9]:
def train(model, data_loader, optimizer, scheduler, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in data_loader:
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['input_ids'])
            loss = outputs.loss
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        print(f"Epoch {epoch + 1}/{num_epochs} Loss: {total_loss / len(data_loader)}")


8. Set Up Optimizer and Scheduler, Train the Model with Multiple Epochs

In [ ]:
epochs_list = [20, 60, 70]  # Training for 20, 60, 70 epochs
for epochs in epochs_list:
    optimizer = AdamW(model.parameters(), lr=5e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=epochs * len(data_loader))
    print(f"\nTraining for {epochs} epochs")
    train(model, data_loader, optimizer, scheduler, num_epochs=epochs)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Training for 20 epochs
Epoch 1/20 Loss: 6.453305244445801
Epoch 2/20 Loss: 3.8783247470855713
Epoch 3/20 Loss: 2.0503785610198975
Epoch 4/20 Loss: 1.346771001815796
Epoch 5/20 Loss: 1.2508608102798462
Epoch 6/20 Loss: 1.094342827796936
Epoch 7/20 Loss: 1.0828231573104858
Epoch 8/20 Loss: 1.0572112798690796
Epoch 9/20 Loss: 0.963986337184906
Epoch 10/20 Loss: 0.866942822933197
Epoch 11/20 Loss: 0.7718837261199951
Epoch 12/20 Loss: 0.7688116431236267
Epoch 13/20 Loss: 0.6826388239860535
Epoch 14/20 Loss: 0.7231718301773071
Epoch 15/20 Loss: 0.7626580595970154
Epoch 16/20 Loss: 0.5730125308036804
Epoch 17/20 Loss: 0.5958646535873413
Epoch 18/20 Loss: 0.6005570888519287
Epoch 19/20 Loss: 0.6370999813079834
Epoch 20/20 Loss: 0.5325086712837219

Training for 60 epochs


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/60 Loss: 0.5235937237739563
Epoch 2/60 Loss: 0.39546406269073486
Epoch 3/60 Loss: 0.3022901117801666
Epoch 4/60 Loss: 0.23461270332336426
Epoch 5/60 Loss: 0.2198304384946823
Epoch 6/60 Loss: 0.1391581892967224
Epoch 7/60 Loss: 0.0822477862238884
Epoch 8/60 Loss: 0.1772851049900055
Epoch 9/60 Loss: 0.039390869438648224
Epoch 10/60 Loss: 0.03356129676103592
Epoch 11/60 Loss: 0.016887908801436424
Epoch 12/60 Loss: 0.027275577187538147
Epoch 13/60 Loss: 0.013647408224642277
Epoch 14/60 Loss: 0.023970413953065872
Epoch 15/60 Loss: 0.07660280168056488
Epoch 16/60 Loss: 0.018887823447585106
Epoch 17/60 Loss: 0.011161288246512413
Epoch 18/60 Loss: 0.015538325533270836
Epoch 19/60 Loss: 0.0058314986526966095
Epoch 20/60 Loss: 0.008851838298141956
Epoch 21/60 Loss: 0.006065582390874624
Epoch 22/60 Loss: 0.007947484962642193
Epoch 23/60 Loss: 0.003908084239810705
Epoch 24/60 Loss: 0.044031016528606415
Epoch 25/60 Loss: 0.005682455841451883
Epoch 26/60 Loss: 0.0685635358095169
Epoch 27/60 

9. Define Text Generation Function

In [ ]:
def generate_text(seed_text, max_length=50):
    model.eval()
    input_ids = tokenizer.encode(seed_text, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7, top_k=50)
    return tokenizer.decode(output[0], skip_special_tokens=True)


10. Generate Text with Initial Seed

In [ ]:
seed_text = "Once upon a time"
generated_text = generate_text(seed_text)
print("Generated Text:\n", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandparents. On her way through the


Experimenting and Improving the Model by large dataset and hyper tune parameter.

1. Try Different Learning Rates

In [ ]:
learning_rates = [5e-5, 3e-5, 1e-4]  # Suggested learning rates to try

for lr in learning_rates:
    print(f"\nTraining with Learning Rate: {lr}")
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=20 * len(data_loader))  # For example, 20 epochs
    train(model, data_loader, optimizer, scheduler, num_epochs=20)  # Testing with 20 epochs for each learning rate



Training with Learning Rate: 5e-05
Epoch 1/20 Loss: 9.206205140799284e-05
Epoch 2/20 Loss: 0.00013883096107747406
Epoch 3/20 Loss: 8.243675256380811e-05
Epoch 4/20 Loss: 5.487834641826339e-05
Epoch 5/20 Loss: 8.417267235927284e-05
Epoch 6/20 Loss: 0.00010850480612134561
Epoch 7/20 Loss: 0.00012759410310536623
Epoch 8/20 Loss: 0.057395026087760925
Epoch 9/20 Loss: 8.167978376150131e-05
Epoch 10/20 Loss: 0.05726826563477516
Epoch 11/20 Loss: 6.209406274138018e-05
Epoch 12/20 Loss: 5.426246207207441e-05
Epoch 13/20 Loss: 8.650391828268766e-05
Epoch 14/20 Loss: 0.05543496087193489
Epoch 15/20 Loss: 4.083941166754812e-05
Epoch 16/20 Loss: 4.8742211220087484e-05
Epoch 17/20 Loss: 8.840572991175577e-05
Epoch 18/20 Loss: 8.497682574670762e-05
Epoch 19/20 Loss: 9.706117998575792e-05
Epoch 20/20 Loss: 0.00012502656318247318

Training with Learning Rate: 3e-05
Epoch 1/20 Loss: 8.785501995589584e-05
Epoch 2/20 Loss: 0.00018074610852636397
Epoch 3/20 Loss: 0.056997623294591904
Epoch 4/20 Loss: 8.5

2. Experiment with Batch Size

In [ ]:
batch_sizes = [1, 2, 4]  # Batch sizes to test

for batch_size in batch_sizes:
    print(f"\nTraining with Batch Size: {batch_size}")
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=20 * len(data_loader))  # For example, 20 epochs
    train(model, data_loader, optimizer, scheduler, num_epochs=20)  # Testing with 20 epochs for each batch size



Training with Batch Size: 1
Epoch 1/20 Loss: 6.0172402299940586e-05
Epoch 2/20 Loss: 4.978180368198082e-05
Epoch 3/20 Loss: 0.0002669463574420661
Epoch 4/20 Loss: 0.0016158678336068988
Epoch 5/20 Loss: 3.118315726169385e-05
Epoch 6/20 Loss: 4.1142062400467694e-05
Epoch 7/20 Loss: 3.593585279304534e-05
Epoch 8/20 Loss: 2.6741474357550032e-05
Epoch 9/20 Loss: 5.239596794126555e-05
Epoch 10/20 Loss: 0.04887615889310837
Epoch 11/20 Loss: 2.8301074053160846e-05
Epoch 12/20 Loss: 3.0371593311429024e-05
Epoch 13/20 Loss: 0.04820052906870842
Epoch 14/20 Loss: 0.04410998150706291
Epoch 15/20 Loss: 4.120239827898331e-05
Epoch 16/20 Loss: 3.796975579462014e-05
Epoch 17/20 Loss: 7.630539039382711e-05
Epoch 18/20 Loss: 2.8483747883001342e-05
Epoch 19/20 Loss: 3.18808488373179e-05
Epoch 20/20 Loss: 2.525188392610289e-05

Training with Batch Size: 2
Epoch 1/20 Loss: 4.673516014008783e-05
Epoch 2/20 Loss: 6.085267887101509e-05
Epoch 3/20 Loss: 0.04772515222430229
Epoch 4/20 Loss: 9.15202617761679e-05

3. Experiment with max_length in Text Generation

In [ ]:
seed_text = "Once upon a time"
max_lengths = [50, 100, 150]  # Different max lengths for generation

for max_len in max_lengths:
    print(f"\nGenerating text with max_length: {max_len}")
    generated_text = generate_text(seed_text, max_length=max_len)
    print("Generated Text:\n", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generating text with max_length: 50


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her aunt. On her way through the

Generating text with max_length: 100


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandma. On her way through the forest, she met a big bad wolf who wanted to eat her.

Generating text with max_length: 150
Generated Text:
 Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandfather. On her way through the forest, she met a big bad wolf who wanted to eat her.
